In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from gtda.pipeline import Pipeline
from gtda.time_series import Resampler
from gtda.diagrams import PersistenceEntropy, Scaler, HeatKernel, BettiCurve
import numpy as np
from gtda.time_series import SingleTakensEmbedding, takens_embedding_optimal_parameters, TakensEmbedding
from sklearn.decomposition import PCA
from gtda.plotting import plot_point_cloud
from gtda.homology import VietorisRipsPersistence
from gtda.metaestimators import CollectionTransformer
import plotly.express as px

In [2]:
def read_files(path, label, limit):
    data = []
    os.chdir(path)
    i = 0
    for file in os.listdir():
        data_df = pd.read_csv(file)
        # fill NaN with an interpolated value
        data_df = data_df.interpolate()
        period = int(len(data_df)/limit)
        periodicSampler = Resampler(period=period)
        print(i,file)
        # resample the files to that all of them are the same length (in entries)
        # NOTE: timestamps are omitted and timesteps are going to be different for each resampled time series!
        index_sampled, signal_sampled = periodicSampler.fit_transform_resample(data_df.index, data_df[label])
        data.append(signal_sampled)
        i += 1
    data_T = list(map(list, zip(*data)))
    data = np.array(data_T)
    df = pd.DataFrame.from_records(data_T)
    return data.T, df

In [3]:
unstable_signals, unstable_df = read_files("/Users/simo/repos/RareEventsDataset/3w_dataset-master/data/data/4/", "P-TPT",3000)

0 WELL-00002_20140126140124.csv
1 WELL-00010_20180427020029.csv
2 WELL-00010_20180425080141.csv
3 WELL-00010_20180425100033.csv
4 WELL-00010_20180423220016.csv
5 WELL-00010_20180422200158.csv
6 WELL-00002_20140122020051.csv
7 WELL-00010_20180426040033.csv
8 WELL-00005_20170625133000.csv
9 WELL-00002_20140126060141.csv
10 WELL-00004_20140806100103.csv
11 WELL-00010_20180424060029.csv
12 WELL-00001_20170317000000.csv
13 WELL-00001_20170317100000.csv
14 WELL-00002_20131215160122.csv
15 WELL-00002_20140122140015.csv
16 WELL-00004_20140805020033.csv
17 WELL-00002_20131215080015.csv
18 WELL-00002_20140126160008.csv
19 WELL-00005_20170626150141.csv
20 WELL-00002_20140126000059.csv
21 WELL-00002_20140124080020.csv
22 WELL-00002_20140123020219.csv
23 WELL-00002_20140125200016.csv
24 WELL-00010_20180427200340.csv
25 WELL-00005_20170626170050.csv
26 WELL-00002_20140126040133.csv
27 WELL-00001_20170319180033.csv
28 WELL-00010_20180425000232.csv
29 WELL-00002_20140119200132.csv
30 WELL-00010_201804

252 WELL-00005_20170626090055.csv
253 WELL-00002_20131215120025.csv
254 WELL-00002_20140122100112.csv
255 WELL-00010_20180423200021.csv
256 WELL-00002_20140124160018.csv
257 WELL-00002_20140122040046.csv
258 WELL-00010_20180428100116.csv
259 WELL-00004_20140805040025.csv
260 WELL-00002_20140115180036.csv
261 WELL-00002_20140109050025.csv
262 WELL-00001_20170317200000.csv
263 WELL-00010_20180428020211.csv
264 WELL-00002_20140125060245.csv
265 WELL-00007_20170518130103.csv
266 WELL-00010_20180416200008.csv
267 WELL-00010_20180428060033.csv
268 WELL-00007_20170518010012.csv
269 WELL-00002_20140124140041.csv
270 WELL-00005_20170624020016.csv
271 WELL-00001_20170317040000.csv
272 WELL-00005_20170625000038.csv
273 WELL-00002_20140119160112.csv
274 WELL-00002_20140119220015.csv
275 WELL-00004_20141117230046.csv
276 WELL-00004_20141117130029.csv
277 WELL-00010_20180423040021.csv
278 WELL-00001_20170318060025.csv
279 WELL-00010_20180425040224.csv
280 WELL-00007_20170518110129.csv
281 WELL-00010

In [4]:
PatoBar = 1/100000
unstable_df = unstable_df.apply(lambda x: x*PatoBar) 
unstable_signals = unstable_signals * PatoBar
unstable_df

,0,1,2,3,4,5,6,7,8,9,...,334,335,336,337,338,339,340,341,342,343
0,162.0611,154.1551,152.8566,153.6380,156.8441,154.7641,163.0213,153.4082,207.7388,161.3322,...,138.4966,145.0387,163.4834,163.5570,157.0394,208.1870,162.9880,155.1893,161.4453,141.0535
1,162.0642,156.0052,154.9020,152.3050,157.1773,153.0175,163.0279,154.5343,207.7618,161.3355,...,138.4944,145.0401,163.4801,163.5586,155.7409,208.1813,162.9880,154.7527,161.4475,141.0468
2,162.0673,153.8678,156.7177,154.1896,155.4077,154.1551,163.0346,155.6605,207.7695,161.3389,...,138.4922,145.0414,163.4768,163.5602,154.4424,208.1927,162.9880,156.8211,161.4497,141.0372
3,162.0703,151.7994,153.8678,156.0741,153.9942,155.2927,163.0412,151.7764,207.7771,161.3422,...,138.4899,145.0427,163.4735,163.5618,153.0405,208.1927,162.9880,156.4648,161.4519,141.0246
4,162.0734,153.5231,153.2013,154.6033,155.0744,157.2463,163.0479,151.7994,207.7848,161.3455,...,138.4877,145.0440,163.4702,163.5634,154.6952,208.1813,162.9880,154.7756,161.4541,141.0120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3479,161.7379,154.4079,153.4082,152.1212,157.0739,156.2695,163.2474,152.9830,208.1985,161.2441,...,143.9162,140.1341,163.7352,161.6913,155.8673,208.1717,163.1908,154.7411,161.1793,142.6395
3480,161.7445,152.9485,153.6610,155.3617,156.1891,155.2927,163.2540,153.7759,208.1755,161.2474,...,143.9096,140.1407,163.7345,161.6846,156.6027,208.1525,163.1975,154.2068,161.1859,142.6395
3481,161.7511,151.6558,155.6375,154.5573,154.3849,153.9597,163.2607,156.5338,208.1985,161.2507,...,143.9029,140.1474,163.7338,161.6780,155.5915,208.1410,163.2041,153.9655,161.1926,142.6395
3482,161.7578,152.8394,156.6487,152.8106,154.1781,156.6947,163.2673,153.3622,208.2023,161.2541,...,143.8963,140.1541,163.7330,161.6714,152.6727,208.1640,163.2108,155.1319,161.1992,142.6395


In [ ]:
from random import gauss
from random import seed
from pandas.plotting import autocorrelation_plot
from statsmodels.graphics.gofplots import qqplot

fig_ss_analysis, data = plt.subplots(1, 3, figsize=(17, 3))
freq_units = 0.1/len(unstable_df[331])
print('freq units: %(units)4E Hz' %{"units":(freq_units)})

fig_ss_analysis.suptitle('instabilities BHP stats')
data[0].set_xlabel('time')
data[0].set_ylabel('BHP (Bar)')
data[1].set_xlabel('freq (%(units).2E Hz)' %{"units":(freq_units)})
data[1].set_ylabel('power spectrum')
data[2].set_xlabel('BHP (Bar)')
data[2].set_ylabel('count')

data[1].set_yscale('log')
data[1].set_xscale('log')

ss_BHP = unstable_df[331]

fft_ss = np.fft.rfft(ss_BHP)
fft_ss_abs = np.abs(fft_ss)
power_spectrum_ss = np.square(fft_ss_abs)


data[0].plot(ss_BHP)
data[1].plot(power_spectrum_ss)
ss_BHP.hist()

In [13]:
def batch_analyzer(input_df, stride, max_embedding_dimension, max_time_delay):
    max_time_delay = int(max_time_delay)
    max_embedding_dimension = int(max_embedding_dimension)
    homology_dimensions = (0, 1)
    VRP = VietorisRipsPersistence(homology_dimensions=homology_dimensions)
    pca = PCA(n_components=3)
    PE_signal = PersistenceEntropy()
    PE_norm = PersistenceEntropy(normalize=True)

    entropies = []
    norm_entropies = []
    diagrams = []
    point_clouds_pca = []
    i = 0
    
    print('max time delay:',max_time_delay, 'max dim:',max_embedding_dimension)
    
    for timeserie in input_df:
        
        i += 1
        optimal_time_delay, optimal_embedding_dimension = takens_embedding_optimal_parameters(
            input_df[timeserie], max_time_delay, max_embedding_dimension, stride=stride
            )
        if optimal_embedding_dimension < 3:
            optimal_embedding_dimension = 3
            
        print('analyzing nr.',i, 'progress:',int(100*i/len(input_df.columns)),'%', 'dim',optimal_embedding_dimension,'delay', optimal_time_delay)
    return

In [15]:
batch_analyzer(unstable_df, 3, 20, 130)

max time delay: 130 max dim: 20
analyzing nr. 1 progress: 0 % dim 7 delay 128
analyzing nr. 2 progress: 0 % dim 6 delay 17
analyzing nr. 3 progress: 0 % dim 8 delay 42
analyzing nr. 4 progress: 1 % dim 6 delay 40
analyzing nr. 5 progress: 1 % dim 6 delay 22
analyzing nr. 6 progress: 1 % dim 6 delay 58
analyzing nr. 7 progress: 2 % dim 8 delay 122
analyzing nr. 8 progress: 2 % dim 6 delay 26
analyzing nr. 9 progress: 2 % dim 11 delay 85
analyzing nr. 10 progress: 2 % dim 7 delay 123
analyzing nr. 11 progress: 3 % dim 7 delay 42
analyzing nr. 12 progress: 3 % dim 6 delay 18
analyzing nr. 13 progress: 3 % dim 8 delay 120
analyzing nr. 14 progress: 4 % dim 3 delay 129
analyzing nr. 15 progress: 4 % dim 7 delay 130
analyzing nr. 16 progress: 4 % dim 7 delay 127
analyzing nr. 17 progress: 4 % dim 8 delay 36
analyzing nr. 18 progress: 5 % dim 3 delay 1
analyzing nr. 19 progress: 5 % dim 6 delay 125
analyzing nr. 20 progress: 5 % dim 13 delay 71
analyzing nr. 21 progress: 6 % dim 9 delay 108
a

analyzing nr. 173 progress: 50 % dim 13 delay 69
analyzing nr. 174 progress: 50 % dim 5 delay 28
analyzing nr. 175 progress: 50 % dim 8 delay 128
analyzing nr. 176 progress: 51 % dim 7 delay 109
analyzing nr. 177 progress: 51 % dim 3 delay 1
analyzing nr. 178 progress: 51 % dim 6 delay 78
analyzing nr. 179 progress: 52 % dim 8 delay 104
analyzing nr. 180 progress: 52 % dim 10 delay 34
analyzing nr. 181 progress: 52 % dim 7 delay 52
analyzing nr. 182 progress: 52 % dim 8 delay 114
analyzing nr. 183 progress: 53 % dim 10 delay 26
analyzing nr. 184 progress: 53 % dim 14 delay 56
analyzing nr. 185 progress: 53 % dim 12 delay 70
analyzing nr. 186 progress: 54 % dim 7 delay 130
analyzing nr. 187 progress: 54 % dim 8 delay 125
analyzing nr. 188 progress: 54 % dim 5 delay 127
analyzing nr. 189 progress: 54 % dim 4 delay 127
analyzing nr. 190 progress: 55 % dim 7 delay 130
analyzing nr. 191 progress: 55 % dim 7 delay 123
analyzing nr. 192 progress: 55 % dim 5 delay 110
analyzing nr. 193 progres

analyzing nr. 342 progress: 99 % dim 6 delay 20
analyzing nr. 343 progress: 99 % dim 8 delay 121
analyzing nr. 344 progress: 100 % dim 7 delay 130


In [ ]:
TE = TakensEmbedding(time_delay=125, dimension=8, stride=3)
homology_dimensions = (0, 1)
VRP = VietorisRipsPersistence(homology_dimensions=homology_dimensions)
PE = PersistenceEntropy()
PE_norm = PersistenceEntropy(normalize=True)
Betti = BettiCurve()

unstable_point_cloud  = TE.fit_transform(unstable_signals)
unstable_diagrams = VRP.fit_transform(unstable_point_cloud)
unstable_entropy = PE.fit_transform(unstable_diagrams)
unstable_entropynorm = PE_norm.fit_transform(unstable_diagrams)
unstable_Betti = Betti.fit_transform(unstable_diagrams)

In [ ]:
tmp_entropies = unstable_entropy
Entropy_H0 = []
Entropy_H1 = []
tmp_normalized = unstable_entropynorm
Entropy_H0_norm = []
Entropy_H1_norm = []

for item in tmp_entropies:
    Entropy_H0.append(item[0][0])
    Entropy_H1.append(item[0][1])

for item in tmp_normalized:
    Entropy_H0_norm.append(item[0][0])
    Entropy_H1_norm.append(item[0][1])
    
Entropy_H1_series = pd.Series(Entropy_H1)
Entropy_H1_norm_series = pd.Series(Entropy_H1_norm)

entropies, ts = plt.subplots(2,figsize=(10,8),sharex = True)

entropies.suptitle('instabilities dataset persistent entropies')
ts[1].set_xlabel("# timeseries")
ts[0].set_ylabel('Entropy')
ts[1].set_ylabel('normalized  Entropy')

degrees = 70
plt.xticks(rotation=degrees)

ts[0].plot(Entropy_H1,'r-')
ts[0].plot(Entropy_H1_series.rolling(20).mean(), 'g-')
ts[0].plot(Entropy_H0,'b-')

ts[1].set_ylim([-20,40])
ts[1].plot(Entropy_H1_norm,'r-')
ts[1].plot(Entropy_H1_norm_series.rolling(20).mean(), 'g-')
ts[1].plot(Entropy_H0_norm,'b-')